<a href="https://colab.research.google.com/github/danielsaggau/DICE/blob/master/Cali_DiCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the packages

In [ ]:
!pip install dice-ml
import numpy as np
import timeit
import random
import dice_ml
import pandas as pd
from dice_ml.utils import helpers  # helper functions
from dice_ml import Dice
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import fetch_california_housing

## Loading California Housing data 

In [6]:
housing = fetch_california_housing()
df_housing = pd.DataFrame(housing.data, columns=housing.feature_names)
df_housing['outcome_name'] = pd.Series(housing.target)
df_housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000). see: https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset

In [9]:
continuous_features_cali = df_housing.drop('outcome_name', axis=1).columns.tolist()
target = df_housing['outcome_name']

Splitting test and training set 

In [12]:
datasetX = df_housing.drop('outcome_name', axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0)

In [32]:
categorical_features = x_train.columns.difference(continuous_features_cali)

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_cali),
        ('cat', categorical_transformer, categorical_features)])

In [33]:
regr_cali = Pipeline(steps=[('preprocessor', transformations),
                              ('regressor', RandomForestRegressor())])

model_cali = regr_cali.fit(x_train, y_train)

In [34]:
d_cali = dice_ml.Data(dataframe=df_housing, continuous_features=continuous_features_cali, outcome_name='outcome_name')
m_cali = dice_ml.Model(model=model_cali, backend="sklearn", model_type='regressor')
exp_genetic_cali = Dice(d_cali, m_cali, method="genetic")

In [35]:
query_instances_cali = x_train[17:19]
query_instances_cali

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
20392,6.0145,14.0,6.156749,1.082729,1858.0,2.696662,34.23,-118.87
20246,3.6413,40.0,4.604736,1.041894,1318.0,2.400729,34.27,-119.26


# Generating Counterfactuals

 Let's see which features need to change to achieve a value in the desired range.

In [46]:
genetic_cali = exp_genetic_cali.generate_counterfactuals(query_instances_cali, 
                                                             total_CFs=2, 
                                                             desired_range=[3.6, 3.8],
                                                         features_to_vary=["MedInc", "Population", "AveRooms"]
                                                         )
genetic_cali.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 2/2 [00:13<00:00,  6.78s/it]

Query instance (original outcome : 3)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,6.0145,14.0,6.156749,1.082729,1858.0,2.696662,34.23,-118.87,3.12278



Diverse Counterfactual set (new outcome: [3.6, 3.8])


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,6.4978,-,6.7,1.1,1928.0,2.7,-,-,3.6419718
0,6.8189,-,7.7,1.1,1913.0,2.7,-,-,3.665472


Query instance (original outcome : 2)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,3.6413,40.0,4.604736,1.041894,1318.0,2.400729,34.27,-119.26,2.27062



Diverse Counterfactual set (new outcome: [3.6, 3.8])


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome_name
0,6.0194,-,5.7,1.0,1358.0,2.4,-,-,3.676750700000001
